In [ ]:
import asyncio
import sys
import warnings
from typing import Dict, Tuple

import pandas as pd
from scipy import stats
from sklearn.linear_model import LinearRegression

from utilities.data_manager import ExchangeDataManager

if sys.platform == 'win32':
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
        
warnings.filterwarnings('ignore')


In [ ]:
def load_equity_from_csv(csv_path: str = "equity.csv") -> pd.Series:
    """
    Load portfolio equity data from CSV file.
    
    Args:
        csv_path: Path to the CSV file containing equity data
        
    Returns:
        pandas Series with datetime index and equity values
    """
    try:
        df = pd.read_csv(csv_path)
        
        df['day'] = pd.to_datetime(df['day'])
        
        equity_series = pd.Series(
            data=df['wallet'].values,
            index=df['day'],
            name='portfolio_equity'
        )
        
        return equity_series
        
    except FileNotFoundError:
        raise FileNotFoundError(f"Equity CSV file not found at: {csv_path}")
    except KeyError as e:
        raise KeyError(f"Required column missing in CSV: {e}")
    except Exception as e:
        raise Exception(f"Error loading equity CSV: {e}")


async def analyze_portfolio_alpha_beta(
    portfolio_equity: pd.Series,
    benchmark: Dict[str, str] = {"symbol": "BTC/USDT", "exchange": "binance"}
) -> None:
    """
    Complete alpha-beta analysis of a portfolio against benchmark.
    
    Args:
        portfolio_equity: Portfolio equity curve (pandas Series with datetime index)
        benchmark: Dictionary with benchmark configuration
        
    Returns:
        Dictionary with complete analysis results
    """
    if not isinstance(portfolio_equity.index, pd.DatetimeIndex):
        raise ValueError("Portfolio equity must have a datetime index")
    
    portfolio_returns = _ensure_daily_returns(portfolio_equity)
    
    if len(portfolio_returns) < 60:
        raise ValueError("Portfolio must have at least 60 days of data for reliable analysis")
    
    benchmark_returns = await _get_benchmark_equity(
        start_date=portfolio_equity.index.min(),
        end_date=portfolio_equity.index.max(),
        benchmark=benchmark
    )
    portfolio_days = (portfolio_returns.index.max() - portfolio_returns.index.min()).days + 1
    benchmark_days = (benchmark_returns.index.max() - benchmark_returns.index.min()).days + 1
    
    aligned_data = pd.DataFrame({
        'portfolio': portfolio_returns,
        'benchmark': benchmark_returns
    }).dropna()
    
    aligned_days = (aligned_data.index.max() - aligned_data.index.min()).days + 1
   
    alpha, beta, reg_stats = _calculate_alpha_beta_regression(
        aligned_data['portfolio'], aligned_data['benchmark']
    )
    alpha_annual = _annualize_daily_alpha(alpha)
    
    interpretations = _interpret_results(beta, alpha_annual, reg_stats)
    
    results = {
        'analysis_period': {
            'start': aligned_data.index.min(),
            'end': aligned_data.index.max()
        },
        'portfolio_dates': f"{portfolio_returns.index.min().date()} to {portfolio_returns.index.max().date()}",
        'portfolio_span': f"{len(portfolio_returns)} points, {portfolio_days} days",
        'benchmark_dates': f"{benchmark_returns.index.min().date()} to {benchmark_returns.index.max().date()}",
        'benchmark_span': f"{len(benchmark_returns)} points, {benchmark_days} days", 
        'aligned_dates': f"{aligned_data.index.min().date()} to {aligned_data.index.max().date()}",
        'aligned_span': f"{len(aligned_data)} points, {aligned_days} days",
        'regression_results': {
            'alpha_daily': alpha,
            'alpha_annual': alpha_annual,
            'beta': beta
        },
        'interpretations': interpretations
    }
    
    _print_results(results)

def _calculate_returns(price_series: pd.Series) -> pd.Series:
    """Calculate returns from price series."""
    returns = price_series.pct_change()
    return returns.dropna()


async def _get_benchmark_equity(
    start_date: pd.Timestamp,
    end_date: pd.Timestamp,
    benchmark: Dict[str, str]
) -> pd.Series:
    """
    Get benchmark equity data, ensuring complete coverage for the requested period.
    
    This function properly handles missing/incomplete data by using the data_manager's
    built-in logic to detect and download missing data.
    
    Args:
        start_date: Start date for benchmark data
        end_date: End date for benchmark data  
        benchmark: Dictionary with benchmark configuration
        
    Returns:
        Daily returns series for the benchmark
        
    Raises:
        ValueError: If insufficient data is available after download attempts
    """
    manager = ExchangeDataManager(exchange_name=benchmark["exchange"], path_download="./database")
    symbol = benchmark['symbol']
    timeframe = "1d"
    
    extended_start = (start_date - pd.Timedelta(days=30)).strftime('%Y-%m-%d %H:%M:%S')
    extended_end = (end_date + pd.Timedelta(days=30)).strftime('%Y-%m-%d %H:%M:%S')
    
    try:
        await manager.download_data(
            coins=[symbol], 
            intervals=[timeframe], 
            start_date=extended_start, 
            end_date=extended_end
        )
        
        try:
            bench_df = manager.load_data(symbol, timeframe, start_date=extended_start, end_date=extended_end)
        except FileNotFoundError:
            raise ValueError(f"Unable to obtain benchmark data for {symbol} from {benchmark['exchange']}")
        
        if bench_df.empty:
            raise ValueError(f"No benchmark data available for {symbol} in the requested period")
        
        benchmark_series = bench_df['close']
        benchmark_returns = _calculate_returns(benchmark_series)
        
    finally:
        if hasattr(manager, 'exchange') and manager.exchange:
            await manager.exchange.close()
    
    data_start = benchmark_returns.index.min()
    data_end = benchmark_returns.index.max()
    
    if data_start > start_date or data_end < end_date:
        print(f"⚠️  Warning: Benchmark data coverage ({data_start.date()} to {data_end.date()}) "
              f"is incomplete for requested period ({start_date.date()} to {end_date.date()})")
    
    return benchmark_returns


def _ensure_daily_returns(equity_curve: pd.Series) -> pd.Series:
    """
    Ensure the equity curve is converted to daily returns regardless of input frequency.
    
    This function will:
    1. Resample any frequency (hourly, minute, etc.) to daily using the last value of each day
    2. Calculate daily returns from the resampled equity values
    
    Args:
        equity_curve: Portfolio equity curve with any frequency
        
    Returns:
        Daily returns series
    """
    if not isinstance(equity_curve.index, pd.DatetimeIndex):
        raise ValueError("Equity curve index must be datetime")
    
    daily_equity = equity_curve.resample('D').last().dropna()
    
    daily_returns = daily_equity.pct_change().dropna()
    
    return daily_returns


def _calculate_alpha_beta_regression(
    portfolio_returns: pd.Series, 
    market_returns: pd.Series
) -> Tuple[float, float, Dict]:
    """Calculate alpha and beta using linear regression."""
    aligned_data = pd.DataFrame({
        'portfolio': portfolio_returns,
        'market': market_returns
    }).dropna()
    
    if len(aligned_data) < 10:
        raise ValueError("Need at least 10 data points for reliable regression")
    
    portfolio_aligned = aligned_data['portfolio'].values
    market_aligned = aligned_data['market'].values.reshape(-1, 1)
    
    model = LinearRegression()
    model.fit(market_aligned, portfolio_aligned)
    
    alpha = model.intercept_
    beta = model.coef_[0]
    r_squared = model.score(market_aligned, portfolio_aligned)
    
    slope, intercept, r_value, p_value, std_err = stats.linregress(
        market_aligned.flatten(), portfolio_aligned
    )
    
    statistics = {
        'r_squared': r_squared,
        'correlation': r_value,
        'observations': len(aligned_data)
    }
    
    return alpha, beta, statistics


def _annualize_daily_alpha(alpha: float) -> float:
    """Annualize daily alpha by multiplying by trading days."""
    return alpha * 365


def _interpret_results(beta: float, alpha_annual: float, reg_stats: Dict) -> Dict[str, str]:
    """Provide human-readable interpretations."""
    interpretations = {}
    
    if beta > 1.2:
        interpretations['beta'] = "High beta - Amplifies market movements significantly"
    elif beta > 1.0:
        interpretations['beta'] = "Moderate beta - Moves more than market"
    elif beta > 0.8:
        interpretations['beta'] = "Market-like beta - Similar to market"
    elif beta > 0.3:
        interpretations['beta'] = "Defensive beta - Less volatile than market"
    else:
        interpretations['beta'] = "Low beta - Weakly correlated to market"
    
    if alpha_annual > 0.05:
        interpretations['alpha'] = "Significant positive alpha - Strategy adds value"
    elif alpha_annual < -0.05:
        interpretations['alpha'] = "Significant negative alpha - Strategy destroys value"
    else:
        interpretations['alpha'] = "Neutral alpha - No significant value added"
    
    return interpretations


def _print_results(results: Dict):
    """Print analysis results nicely."""
    print("\n" + "="*60)
    print("📊 ALPHA-BETA ANALYSIS RESULTS")
    print("="*60)
    
    portfolio_span = results.get('portfolio_span', 'N/A')
    benchmark_span = results.get('benchmark_span', 'N/A') 
    aligned_span = results.get('aligned_span', 'N/A')
    
    print(f"Portfolio Period: {results.get('portfolio_dates', 'N/A')} ({portfolio_span})")
    print(f"Benchmark Period:: {results.get('benchmark_dates', 'N/A')} ({benchmark_span})")
    print(f"Aligned Period:: {results.get('aligned_dates', 'N/A')} ({aligned_span})")
    
    reg = results['regression_results']
    print("\n📈 ALPHA-BETA METRICS:")
    print(f"   • Beta: {reg['beta']:.3f}")
    print(f"   • Daily Alpha: {reg['alpha_daily']*100:.3f}%")
    print(f"   • Annual Alpha: {reg['alpha_annual']*100:.1f}%")
    
    print("\n💡 INTERPRETATIONS:")
    for key, interpretation in results['interpretations'].items():
        print(f"   • {interpretation}")

In [ ]:
file_path = "multi_envelope_equity.csv"

portfolio_equity = load_equity_from_csv(file_path)
results = await analyze_portfolio_alpha_beta(
    portfolio_equity=portfolio_equity,
    benchmark={"symbol": "BTC/USDT", "exchange": "binance"}
)
